## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

import sys
sys.path.append('..')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 

city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Country
0,0,Mataura,-46.19,168.86,46.00,97,96,1.99,light rain,NZ
1,1,Salinopolis,-0.61,-47.36,85.03,67,43,11.21,scattered clouds,BR
2,2,Pevek,69.70,170.31,38.48,88,100,3.47,overcast clouds,RU
3,3,Luanda,-8.84,13.23,75.20,83,20,9.17,few clouds,AO
4,4,Ancud,-41.87,-73.82,48.00,100,90,3.36,overcast clouds,CL


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


#min_temp = 75.0
#max_temp = 90.0


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Country
1,1,Salinopolis,-0.61,-47.36,85.03,67,43,11.21,scattered clouds,BR
3,3,Luanda,-8.84,13.23,75.20,83,20,9.17,few clouds,AO
10,10,Albany,42.60,-73.97,81.00,55,0,5.26,clear sky,US
12,12,Puerto Ayora,-0.74,-90.35,75.20,53,75,18.34,broken clouds,EC
13,13,Gat,31.61,34.76,88.00,47,0,8.57,clear sky,IL
14,14,Cabo San Lucas,22.89,-109.91,82.99,94,75,4.70,broken clouds,MX
15,15,Asau,46.43,26.40,82.40,48,49,9.17,scattered clouds,RO
18,18,Celina,40.55,-84.57,79.00,60,1,1.95,clear sky,US
19,19,Itamaraca,-7.75,-34.83,84.20,66,40,14.99,scattered clouds,BR
20,20,Mahina,-17.51,-149.49,75.20,78,40,3.36,shower rain,PF


In [5]:
# 4a. Determine if there are any empty rows.
city_data_df.isnull().sum()

City_ID        0
City           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
Country        0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# already did above?

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_data_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Salinopolis,BR,85.03,scattered clouds,-0.61,-47.36,
3,Luanda,AO,75.20,few clouds,-8.84,13.23,
10,Albany,US,81.00,clear sky,42.60,-73.97,
12,Puerto Ayora,EC,75.20,broken clouds,-0.74,-90.35,
13,Gat,IL,88.00,clear sky,31.61,34.76,
14,Cabo San Lucas,MX,82.99,broken clouds,22.89,-109.91,
15,Asau,RO,82.40,scattered clouds,46.43,26.40,
18,Celina,US,79.00,clear sky,40.55,-84.57,
19,Itamaraca,BR,84.20,scattered clouds,-7.75,-34.83,
20,Mahina,PF,75.20,shower rain,-17.51,-149.49,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    params["location"] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Salinopolis,BR,85.03,scattered clouds,-0.61,-47.36,Hotel Salinópolis
3,Luanda,AO,75.20,few clouds,-8.84,13.23,EPIC SANA Luanda
12,Puerto Ayora,EC,75.20,broken clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
13,Gat,IL,88.00,clear sky,31.61,34.76,רגע בנחת
14,Cabo San Lucas,MX,82.99,broken clouds,22.89,-109.91,Hotel Tesoro Los Cabos
15,Asau,RO,82.40,scattered clouds,46.43,26.40,Casa Mika Ciobanus
18,Celina,US,79.00,clear sky,40.55,-84.57,Americas Best Value Inn Celina
19,Itamaraca,BR,84.20,scattered clouds,-7.75,-34.83,Special K's Bed & Breakfast
20,Mahina,PF,75.20,shower rain,-17.51,-149.49,Guest Beach Bungalow Tahiti
22,Malegaon,IN,77.65,overcast clouds,20.55,74.53,Dinesh Lodge


In [13]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
#locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))